In [1]:
using Flux
using BSON: @save, @load
using BSON
using Markdown
using SparseArrays
using BlockArrays
using Mosek
using MosekTools
using LinearAlgebra
using Printf
using JuMP
using MAT
using ReinforcementLearning
using FurutaRL
using IntervalSets
using StableRNGs
using CircularArrayBuffers
using CUDA

In [2]:
# @load "MNIST_models/001_pgd_model.BSON" pgd_model
# model = pgd_model
@load "stat_agents_4/01500pgd05.bson" model
model = model.policy.behavior_actor.model
Ws = []
for i in 1:length(Flux.params(model))
    if i % 2 == 1
        push!(Ws, Flux.params(model)[i])
    end
end
# W1 = Flux.params(model)[1]
# W2 = Flux.params(model)[3]
# W3 = Flux.params(model)[5]
# # W4 = Flux.params(model)[7]
# # W5 = Flux.params(model)[9]
# # W6 = Flux.params(model)[11]

# b1 = Flux.params(model)[2]
# b2 = Flux.params(model)[4]
# b3 = Flux.params(model)[6]
# # b4 = Flux.params(model)[8]
# # b5 = Flux.params(model)[10]
# # b6 = Flux.params(model)[12]

# Ws = [W1, W2, W3]#, W4, W5, W6]
N = [size(w)[1] for w in Ws]
M = [size(w)[2] for w in Ws]
println(N)
println(M)

A = BlockArray(zeros(sum(N[1:end-1]), sum(M)), N[1:end-1], M)
for i in 1:length(Ws)-1
    setblock!(A, Ws[i], i,i)
end
# setblock!(A, W1, 1,1)
# setblock!(A, W2, 2,2)
# setblock!(A, W3, 3,3)
# setblock!(A, W4, 4,4)
# setblock!(A, W5, 5,5)
A = sparse(A)

B = BlockArray(zeros(sum(N[1:end-1]), sum(M)), N[1:end-1], M)
for i in 1:length(Ws)-1
    setblock!(B, Matrix(I, N[i], N[i]), i,i+1)
end
# setblock!(B, Matrix(I, N[1], N[1]), 1,2)
# setblock!(B, Matrix(I, N[2], N[2]), 2,3)
# setblock!(B, Matrix(I, N[3], N[3]), 3,4)
# setblock!(B, Matrix(I, N[4], N[4]), 4,5)
# setblock!(B, Matrix(I, N[5], N[5]), 5,6)
B = sparse(B);

[24, 48, 72, 96, 128, 1]
[5, 24, 48, 72, 96, 128]


In [3]:
n = sum(N[1:end-1])
m = sum(M)
Vs = []
V0 = -sparse(I, M[1], M[1])
V0i, V0j, V0v = findnz(V0)
V0 = sparse(V0i, V0j, V0v, sum(M), sum(M))
push!(Vs, sparse(LowerTriangular(V0)))
for i in 1:n
    T = sparse([i], [i], [1.0], n, n)
    Vi = B'*T*A + A'*T*B - 2*B'*T*B
    push!(Vs, sparse(LowerTriangular(Vi)))
end
W = [zeros(sum(M[1:end-1]), sum(M));
    zeros(M[end], sum(M[1:end-1])) Ws[end]'*Ws[end]]
W = sparse(LowerTriangular(W));
V0

373×373 SparseMatrixCSC{Int64,Int64} with 5 stored entries:
  [1, 1]  =  -1
  [2, 2]  =  -1
  [3, 3]  =  -1
  [4, 4]  =  -1
  [5, 5]  =  -1

In [4]:
barai, baraj, baraval = [], [], []

for i in 1:n+1
    ai, aj, av = findnz(-Vs[i])
    push!(barai, ai)
    push!(baraj, aj)
    push!(baraval, av)
end

barci, barcj, barcval = findnz(W)

blc = [0.0 for k in 1:n+1]
blc[1] = 1.0
buc = [0.0 for k in 1:n+1]
buc[1] = 1.0
bkc = [MSK_BK_FX for k in 1:n+1]

numcon = length(bkc)

barvardim = [m];

In [ ]:
printstream(msg::String) = print(msg)

t = maketask() do task
    putstreamfunc(task, MSK_STREAM_LOG, printstream)
    
    appendcons(task, numcon)
    appendbarvars(task, barvardim)
    putconboundslice(task, 1, numcon+1, bkc, blc, buc)
    
    symc = appendsparsesymmat(task, barvardim[1], barci, barcj, barcval)
    putbarcj(task, 1, [symc], [1.0])
    
    idxs = []
    for i in 1:n+1
        idx = appendsparsesymmat(task, barvardim[1], barai[i], baraj[i], baraval[i])
        push!(idxs, idx)
    end
    
    for k in 1:n+1
        putbaraij(task, k, 1, [idxs[k]], [1.0])
    end
    
    putobjsense(task, MSK_OBJECTIVE_SENSE_MAXIMIZE)
    
    optimize(task)
    #solutionsummary(task, MSK_STREAM_MSG)
#     prosta = getprosta(task, MSK_SOL_ITR)
#     solsta = getsolsta(task, MSK_SOL_ITR)
# 
#     if solsta == MSK_SOL_STA_OPTIMAL
#         # Output a solution
#         #xx = getxx(task,MSK_SOL_ITR)
#         #barx = getbarxj(task,MSK_SOL_ITR, 1)
#         #barx = reshape(barx, (n,n))
#         #matx = zeros(n,n)
# #         k = 1
# #         for j in 1:n
# #             for i in j:n
# #                 matx[i,j] = barx[k]
# #                 k += 1
# #             end
# #         end
# #         Z = matx'+matx - Diagonal(matx)
# #         yy = getsuc(task,MSK_SOL_ITR)

# #         @printf("Optimal solution: \n  xx = %s\n  barx = %s\n  yy = %s\n", xx',barx', yy')
# #         #@printf("Optimal solution: \n  yy = %s\n", yy')
#         #display(size(Z))
#         ρ = getdualobj(task, MSK_SOL_ITR)
#         x = getsuc(task, MSK_SOL_ITR)
#         @printf("Optimal solution: \n  dual value = %s\n  x = %s", ρ, x)
#     elseif solsta == MSK_SOL_STA_DUAL_INFEAS_CER
#         println("Primal or dual infeasibility.\n")
#     elseif solsta == MSK_SOL_STA_PRIM_INFEAS_CER
#         println("Primal or dual infeasibility.\n")
#     elseif  solsta == MSK_SOL_STA_UNKNOWN
#         println("Unknown solution status")
#     else
#         println("Other solution status")
#     end
end

Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 369             
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

BASELINE4: sqrt(1303) = 36,10

stat_agents_4:
mean = 46.914, std = 12.856
00250pgd01: sqrt(2714) = 52,10
00250pgd02: sqrt(2501) = 50,01
00250pgd03: sqrt(928) = 30,46
00250pgd04: sqrt(1470) = 38,34
00250pgd05: sqrt(4053) = 63,66

mean = 39.790, std = 14.139
00500pgd01: sqrt(844) = 29,05
00500pgd02: sqrt(1127) = 33,57
00500pgd03: sqrt(3819) = 61,80
00500pgd04: sqrt(2105) = 45,88
00500pgd05: sqrt(821) = 28,65

mean = 54.588, std = 9.153
00750pgd01: sqrt(2172) = 46,60
00750pgd02: sqrt(1863) = 43,16
00750pgd03: sqrt(4068) = 63,78
00750pgd04: sqrt(3734) = 61,11
00750pgd05: sqrt(3398) = 58,29

mean = 40.522, std = 11.377
01000pgd01: sqrt(1361) = 36,89
01000pgd02: sqrt(1281) = 35,79
01000pgd03: sqrt(3107) = 55,74
01000pgd04: sqrt(2093) = 47,75
01000pgd05: sqrt(699) = 26,44

mean = 43.238, std = 7.320
01250pgd01: sqrt(2743) = 52,37
01250pgd02: sqrt(1517) = 38,95
01250pgd03: sqrt(2487) = 49,87
01250pgd04: sqrt(1495) = 38,67
01250pgd05: sqrt(1320) = 36,33

mean = 43.980, std = 19.892 (Temp values)
01500pgd01: sqrt(1834) = 42,83
01500pgd02: sqrt(477) = 21,84
01500pgd03: sqrt(5046) = 71,04
01500pgd04: sqrt(3065) = 55,36
01500pgd05: sqrt(831) = 28,83

MNIST:
baseline(model): sqrt(78331) = 279,88
001_pgd_model: sqrt(54336) = 233,10
005_pgd_model: sqrt(6576) = 81,09
01_pgd_model: sqrt(19295) = 138,91
01_pgd_model: sqrt(12265) = 110,75
015_pgd_model: sqrt(49879) = 223,34
015_pgd_model02: sqrt(46272) = 215,11

In [5]:
using JuMP
using MosekTools
using SparseArrays

for n in [ 10, 20, 50, 100, 200, 400 ]

    model = Model(Mosek.Optimizer)
    MOI.set(model, MOI.Silent(), true) 
    
    @variables model begin
        P[1:n]
    end

    s = sum(P)
    M = P |> Diagonal #|> Matrix
    M = sparse(M)

    @objective model Min s
    @SDconstraint model M>=one(M);

    @show n
    @time optimize!(model)
    @show termination_status(model)
end


n = 10
  8.794401

LoadError: InterruptException:

In [4]:
display(norm(W6*W5*W4*W3*W2*W1))
display(norm(W6)*norm(W5)*norm(W4)*norm(W3)*norm(W2)*norm(W1))

6.292103f0

162.11064f0

In [9]:
using JuMP
using MosekTools
using LinearAlgebra
using SparseArrays

n = 200
model = Model(Mosek.Optimizer)
    
@variables model begin
    P[1:n]
end

s = sum(P)
M = P |> Diagonal |> Matrix
M = sparse(M)

@objective model Min s
@SDconstraint model M >= sparse(Matrix(I,n,n))

@time optimize!(model)
@show termination_status(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 20100           
  Cones                  : 0               
  Scalar variables       : 200             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimizat

OPTIMAL::TerminationStatusCode = 1

In [10]:
using JuMP
using MosekTools
using LinearAlgebra

n = 200
model = Model(Mosek.Optimizer)
@variable(model, P[1:n])
@variable(model, M[1:n, 1:n], PSD)
@constraint(model, [i=1:n], M[i, i] == P[i] - 1)
@constraint(model, [i=1:n-1], M[i+1,i] == P[i])
@objective(model, Min, sum(P))
@time optimize!(model)
@show termination_status(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 399             
  Cones                  : 0               
  Scalar variables       : 200             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

OPTIMAL::TerminationStatusCode = 1

In [11]:
using JuMP
using MosekTools
using LinearAlgebra
using SparseArrays

n = 100
model = Model(Mosek.Optimizer)
@variable(model, P[1:n])
@variable(model, M[1:n, 1:n], PSD)
Pmat = sparse(Bidiagonal(P, P[1:end-1], :L) - 1.0*I(n))
Pi,Pj,Pv = findnz(Pmat)
@constraint(model, [i in 1:size(Pi,1)], M[Pi[i], Pj[i]] == Pv[i])
@objective(model, Min, sum(P))
findnz(Pmat)[1]
@time optimize!(model)
@show termination_status(model)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 199             
  Cones                  : 0               
  Scalar variables       : 100             
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

OPTIMAL::TerminationStatusCode = 1

In [12]:
1.0*I(4)

4×4 Diagonal{Float64,Array{Float64,1}}:
 1.0   ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅ 
  ⋅    ⋅    ⋅   1.0

In [3]:
n = sum(N)-1
m = sum(M)
Vs = []
V0 = -sparse(I, M[1], M[1])
V0i, V0j, V0v = findnz(V0)
V0 = sparse(V0i, V0j, V0v, sum(M), sum(M))
for i in 1:n
    T = sparse([i], [i], [1.0], n, n)
    Vi = B'*T*A + A'*T*B - 2*B'*T*B
    push!(Vs, sparse(LowerTriangular(Vi)))
end
W = [zeros(sum(M[1:end-1]), sum(M));
    zeros(M[end], sum(M[1:end-1])) W6'*W6]
W = sparse(W);

In [4]:
opt_model = Model(Mosek.Optimizer)
@variable(opt_model, λ[1:n] >= 0)
@variable(opt_model, ρ)
@variable(opt_model, X[1:m, 1:m], PSD)

M_mat = V0*ρ + W
for i in 1:n
    M_mat += Vs[i]*λ[i]
end

M_mat

373×373 SparseMatrixCSC{GenericAffExpr{Float64,VariableRef},Int64} with 139129 stored entries:
  [1  ,   1]  =  -ρ
  [2  ,   1]  =  0
  [3  ,   1]  =  0
  [4  ,   1]  =  0
  [5  ,   1]  =  0
  [6  ,   1]  =  0.23890239000320435 λ[1]
  [7  ,   1]  =  -0.16233018040657043 λ[2]
  [8  ,   1]  =  -0.5195293426513672 λ[3]
  [9  ,   1]  =  -0.2265883982181549 λ[4]
  [10 ,   1]  =  -0.5657771825790405 λ[5]
  [11 ,   1]  =  -0.48833364248275757 λ[6]
  [12 ,   1]  =  0.24110642075538635 λ[7]
  ⋮
  [361, 373]  =  -0.12313338369131088
  [362, 373]  =  0.1770818531513214
  [363, 373]  =  1.0952225437677043e-7
  [364, 373]  =  0.18936306238174438
  [365, 373]  =  -2.913925212233437e-10
  [366, 373]  =  -0.15717095136642456
  [367, 373]  =  0.0336885005235672
  [368, 373]  =  0.15616025030612946
  [369, 373]  =  -1.0932184935086298e-8
  [370, 373]  =  -0.13415804505348206
  [371, 373]  =  -0.138919860124588
  [372, 373]  =  0.1745382845401764
  [373, 373]  =  -2 λ[368] + 0.18508023023605347

In [18]:
n = 100
model = Model(Mosek.Optimizer)
MOI.set(model, MOI.Silent(), false) 
    
@variables model begin
    P[1:n]
end

s = sum(P)
d = Dict{Tuple{Int64,Int64}, Any}()
for i in 1:n
    d[(i, i)] = P[i]
end
M = JuMP.Containers.SparseAxisArray(d)

JuMP.Containers.SparseAxisArray{Any,2,Tuple{Int64,Int64}} with 100 entries:
  [21, 21]  =  P[21]
  [34, 34]  =  P[34]
  [72, 72]  =  P[72]
  [70, 70]  =  P[70]
  [88, 88]  =  P[88]
  [4, 4  ]  =  P[4]
  [45, 45]  =  P[45]
  [62, 62]  =  P[62]
  [71, 71]  =  P[71]
  [15, 15]  =  P[15]
            ⋮
  [39, 39]  =  P[39]
  [84, 84]  =  P[84]
  [76, 76]  =  P[76]
  [12, 12]  =  P[12]
  [79, 79]  =  P[79]
  [90, 90]  =  P[90]
  [80, 80]  =  P[80]
  [25, 25]  =  P[25]
  [36, 36]  =  P[36]
  [98, 98]  =  P[98]
  [77, 77]  =  P[77]

In [4]:
@load "6msgoodguy_model.bson" model
W1 = Flux.params(model)[1]
W2 = Flux.params(model)[3]
W3 = Flux.params(model)[5]
W4 = Flux.params(model)[7]
W5 = Flux.params(model)[9]
W6 = Flux.params(model)[11]
matwrite("6msgoodguy_model.mat", Dict(
        "W1" => W1, "W2" => W2, "W3" => W3, "W4" => W4, "W5" => W5, "W6" => W6))


\begin{align}
    \min & \hspace{0.5cm} \langle \bar{C}_1, Z_1\rangle + ... + \langle \bar{C}_m, Z_m\rangle + c^\intercal z \\
    \text{s.t.} & \hspace{0.5cm} \begin{bmatrix}
        \langle\bar{a_1}, Z\rangle \\
        ... \\
        \langle\bar{a_n}, Z\rangle
    \end{bmatrix} + Az = b \\
    & \hspace{0.5cm} Z \succeq 0
\end{align}


$\alpha = 0$, $\beta = 1$ gives

\begin{equation}
    \begin{bmatrix} A^\intercal & B^\intercal \end{bmatrix} \begin{bmatrix} 
    -2\alpha \beta T & (\alpha + \beta )T \\
    (\alpha + \beta )T & -2T
    \end{bmatrix} \begin{bmatrix} A \\ B \end{bmatrix} = 
    \begin{bmatrix} A^\intercal & B^\intercal \end{bmatrix} \begin{bmatrix} 
    0 & \beta T \\
    \beta T & -2T
    \end{bmatrix} \begin{bmatrix} A \\ B \end{bmatrix} \\
    = B^\intercal TA + A^\intercal TB - 2B^\intercal TB
\end{equation}

Assuming $T = \text{Diag}\{\lambda_i\}_{i = 1,...,n}$, we can decompose this matrix into

\begin{equation}
    \sum\limits_{i=1}^{n} V_i\lambda_i
\end{equation}

where $V_i$ is found by setting $\lambda_i = 1$ and $ \lambda_j = 0, i \neq j$. Additionally define $V_0$ as

\begin{equation}
    V_0 = \begin{bmatrix}
    -I_{n_0} & 0 & \cdots \\
    0 & 0 & \cdots \\
    \vdots & \vdots & \ddots
    \end{bmatrix}
\end{equation}

Now define $x = [\rho,  \lambda_1, \ldots, \lambda_n]^\intercal$, and $W$ as

\begin{equation}
    W = \begin{bmatrix}
    0 & 0 & \cdots & 0 \\
    0 & 0 & \cdots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \cdots & (W^l)^\intercal W^l
    \end{bmatrix}
\end{equation}

We can now write $M(\rho, T)$ as

\begin{equation}
    M(\rho, T) = W + \sum\limits_{i=0}^{n} V_ix_i
\end{equation}

The constraint $M(\rho, T) \preceq 0 \Leftrightarrow -M(\rho, T) \succeq 0$ can now be written as

\begin{equation}
    A_0 + \sum\limits_{i=1}^{n+1} A_i x_i \succeq 0
\end{equation}

with $A_0 = -W$ and $A_i = -V_{i-1}$, $i > 0$. This lets us write the optimization problem as

\begin{align}
    \min & \hspace{0.5cm} c^\intercal x \\
    \text{s.t.}&\hspace{0.5cm}  A_0 + \sum\limits_{i=1}^{n+1} A_i x_i = X \\
    & \hspace{0.5cm} X \succeq 0
\end{align}

with $c^\intercal = [1, 0, 0, \ldots ] $ The dual to this problem is 

\begin{align}
    \max & \hspace{0.5cm} -\langle A_0, Z\rangle \\
    \text{s.t.}&\hspace{0.5cm}  \langle A_i, Z\rangle = c_i, \hspace{0.1cm} i = 1,\ldots,n+1 \\
    & \hspace{0.5cm} Z \succeq 0
\end{align}